In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 1️⃣ Initialisation de Spark
spark = SparkSession.builder \
    .appName("RiskScoring_LostObjects") \
    .config("spark.driver.memory", "4g") \  # Alloue 4 Go de mémoire
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .getOrCreate()

SyntaxError: unexpected character after line continuation character (4080036407.py, line 11)

In [8]:
# 2️⃣ Définition du schéma explicite
schema = StructType([
    StructField("date", StringType(), True),
    StructField("gc_obo_date_heure_restitution_c", StringType(), True),
    StructField("gc_obo_gare_origine_r_name", StringType(), True),
    StructField("gc_obo_objet_c", StringType(), True),
    StructField("gc_obo_type_c", StringType(), True)  # Colonne cible
])

In [10]:
# 3️⃣ Données d'entraînement (exemple simplifié)
data = [
    {
        "date": "2024-01-01",
        "gc_obo_date_heure_restitution_c": "2024-01-03 10:00",
        "gc_obo_gare_origine_r_name": "Gare de Lyon",
        "gc_obo_objet_c": "Sac noir",
        "gc_obo_type_c": "Bagage"
    },
    {
        "date": "2024-01-05",
        "gc_obo_date_heure_restitution_c": "2024-01-06 14:00",
        "gc_obo_gare_origine_r_name": "Gare Montparnasse",
        "gc_obo_objet_c": "Téléphone",
        "gc_obo_type_c": "Électronique"
    },
    {
        "date": "2024-01-10",
        "gc_obo_date_heure_restitution_c": null,
        "gc_obo_gare_origine_r_name": "Gare du Nord",
        "gc_obo_objet_c": "Portefeuille rouge",
        "gc_obo_type_c": "Accessoire"
    },
    {
        "date": "2024-01-12",
        "gc_obo_date_heure_restitution_c": "2024-01-14 09:00",
        "gc_obo_gare_origine_r_name": "Gare de Lyon",
        "gc_obo_objet_c": "Veste en cuir",
        "gc_obo_type_c": "Vêtement"
    },
    {
        "date": "2024-01-15",
        "gc_obo_date_heure_restitution_c": null,
        "gc_obo_gare_origine_r_name": "Gare Montparnasse",
        "gc_obo_objet_c": "Ordinateur portable",
        "gc_obo_type_c": "Électronique"
    }
]


In [12]:
# 4️⃣ Création du DataFrame d'entraînement
df = spark.createDataFrame(data, schema=schema)

In [14]:
# 5️⃣ Gestion des valeurs manquantes
df = df.fillna({"gc_obo_date_heure_restitution_c": "Inconnu"})

In [16]:
# 6️⃣ Pipeline de traitement des données
# Encodage des variables catégorielles
gare_indexer = StringIndexer(inputCol="gc_obo_gare_origine_r_name", outputCol="gare_index")
objet_indexer = StringIndexer(inputCol="gc_obo_objet_c", outputCol="objet_index")
type_indexer = StringIndexer(inputCol="gc_obo_type_c", outputCol="label")  # Target

In [18]:
# Assemblage des features
assembler = VectorAssembler(inputCols=["gare_index", "objet_index"], outputCol="features")

In [20]:
# Modèle de classification (Random Forest)
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=10)

In [22]:
# Pipeline complet
pipeline = Pipeline(stages=[gare_indexer, objet_indexer, type_indexer, assembler, rf])

In [24]:
# 7️⃣ Entraînement du modèle
model = pipeline.fit(df)

Py4JJavaError: An error occurred while calling o84.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 1.0 failed 1 times, most recent failure: Lost task 2.0 in stage 1.0 (TID 14) (DESKTOP-IT4SRQE executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.